# Configure CLM inputs from template

To launch this notebook interactively in a Jupyter notebook-like browser interface, please click the "Launch Binder" button below. Note that Binder may take several minutes to launch.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/hydroframe/subsettools-binder/HEAD?labpath=subsettools%2Fconfigure_clm_from_template.ipynb)

The CLM model is a land modification of the Community Land Model that is coupled to ParFlow. If you want to complete transient simulations using ParFlow-CLM you will need additional input files specifically for CLM (you can refer to the ParFlow short courses for more information on ParFlow-CLM [here](https://hydroframe.org/parflow-resources)). 

The additional inputs you will need to run a ParFlow-CLM file are: 
1. `CLM Driver File`: This provides additional settings to configure the CLM run which are not included in the ParFlow run script. We will show you how to start from an existing template.
2. `Vegm File`: This file assigns a land cover type to every cell in the model and is formatted specifically for CLM. You can subset this from the national run for your domain. 
3. `Vegp File`: This has all of the parameters for every land cover type. You can use the national vegp file as a template and will only need to modify if you want to change the parameters of a specific landcover type or introduce a new land cover category. 

There are currently two available CLM configurations one for the [ParFlow CONUS1 model](https://www.hydroframe.org/parflow-conus1) and one for the [ParFlow CONUS2 model](https://www.hydroframe.org/parflow-conus2). The biggest difference between these two configurations is just the updated landcover types for the CONUS2 model. **You should use the CLM configuration that matches the grid you have selected for the rest of your subsetting**  

### 1.  Setup 

In all examples you will need to import the following packages and register your pin in order to have access to the HydroData datasets. 

Refer to the [getting started](https://hydroframesubsettools.readthedocs.io/en/latest/getting_started.html) instructions for creating your pin if you have not done this already.

In [1]:
import subsettools as st
import hf_hydrodata as hf
from parflow import Run
import numpy as np

hf.register_api_pin("your_email", "your_pin")

## 2. Configuring_clm with the `config_clm` function 
The `config_clm` function (API reference [here](https://hydroframesubsettools.readthedocs.io/en/latest/autoapi/subsettools/index.html#subsettools.config_clm)) will complete all three of the steps listed above: 
    1. Grabbing a clm-driver file for you to start from and modifying it to match your domain
    2. Subsetting the the vegm file for your subdomain
    3. Grabbing a vegp file for you to use

As with our other subsetting steps we will pass the `ij_box_bounds` (refer to the [Define your subset domain](https://hydroframesubsettools.readthedocs.io/en/latest/examples/definte_subset.html) tutorial for details). We will also need `start` and `end` dates for our ParFlow simulation, a dataset to get our CLM files from and the directory path where the files are going to be written to. 

The function will return a dictionary in which the keys are (“vegp”, “vegm”, “drv_clm”) and the values are file paths where the CLM files were written.

**NOTE:** *If you choose to provide a timezone while setting up a ParFlow simulation, it should be consistent across the functions `subset_press_init`, `subset_forcing` and `config_clm`.* This is an important parameter in CLM that is used to calculate solar zenith angle so its critical that the time zone you provide match your run. 

**NOTE:** The grid you use for your ij bounds must match the grid of the CLM files you are subsetting. 

In [4]:
#Set the ij bounds for the new model
ij_box_bounds, _ = st.define_latlon_domain(latlon_bounds=[[37.91, -91.43], [37.34, -90.63]], grid="conus1")
print(f"bounding box: {ij_box_bounds}") 

#Get the CLM inputs
file_paths = st.config_clm(
    ij_box_bounds, 
    start="2005-10-01", 
    end="2006-10-01", 
    dataset="conus1_baseline_mod",
    write_dir="/path/to/your/write/directory",
)

bounding box: (2285, 436, 2358, 495)
processing vegp
copied vegp
processing vegm
subset vegm
processing drv_clm
copied drv_clmin
edited drv_clmin


## 3. Getting the clm input files with the HydroData API
The `config_clm` function is the easiest way to get all your CLM inputs configured for your domain in one step. However, as with all the other subsetting steps you can also obtain any of the input files directly on their own using the HydroData API. 

To do this we will use the `get_raw_file` function to get the `vegp` and `drv_clm` files. These are small text files and the `get_raw_file` function is going to get them from HydroData and write them to the given directory path. Note that the `get_raw_file` function is going to copy the files as they are - it will not modify them like the `config_clm` function.

**NOTE:** Technically you can also use this approach to get the national `vegm` landcover file however subsetting this file requires a specific approach due to the formatting of the `vegm` file so we highly recommend you use the `config_clm` function above to get the subset for your domain directly.  

In [6]:
# get the vegp file and write it to filepath:
hf.get_raw_file(filepath="/path/to/your/write/directory",
                         dataset="conus1_baseline_mod",
                         file_type="vegp",
)

# get the drv_clm file and write it to filepath:
hf.get_raw_file(filepath="/path/to/your/write/directory",
                         dataset="conus1_baseline_mod",
                         file_type="drv_clm",
)

### 4. Cite the data sources

Please make sure to cite all data sources that you use. We will use the `hf.get_citations` function, which takes as an argument a dataset name and returns citation information about that dataset:

In [3]:
hf.get_citations("conus1_baseline_mod")

'Modern CONUS1 simulations WY 2003-2006\n  Source: https://doi.org/10.5194/gmd-14-7223-2021\n'